In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l1, l2

from tensorflow.keras.layers import Dense, Activation, Dropout, Normalization, Input, Reshape, Conv2D,BatchNormalization, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K
from sklearn.metrics import precision_recall_fscore_support
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score

seed = 31102016 
rng = np.random.RandomState(seed)

SENSOR_CHANNELS = 3
SLIDING_WINDOW_LENGTH = 100

2024-01-05 15:53:48.927606: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 15:53:48.975553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 15:53:48.975602: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 15:53:48.975635: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 15:53:48.983423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 15:53:48.984241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
filename = "dataset/consolidated.csv" # filtered data
# filename = "dataset/consolidated.csv"  # raw consolidated data
dataset = pd.read_csv(filename, index_col='Unnamed: 0')

students = dataset.student_number.unique()

dataset.head(10)

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,student_number,activity,sub_activity,is_stationary,activity_label,sub_activity_label
0,1697605965,0.011963,-0.855774,-0.029846,-9.625000,-0.859375,-5.656250,s1,ascending,breathingNormal,0,0,0
1,1697606005,-0.001709,-0.826233,-0.036194,0.359375,4.953125,-1.656250,s1,ascending,breathingNormal,0,0,0
2,1697606045,-0.058838,-0.933899,-0.032532,3.437500,9.406250,-2.031250,s1,ascending,breathingNormal,0,0,0
3,1697606085,-0.002441,-1.115051,-0.028870,3.078125,6.921875,-5.984375,s1,ascending,breathingNormal,0,0,0
4,1697606125,-0.036621,-1.035217,-0.076477,4.328125,8.125000,0.625000,s1,ascending,breathingNormal,0,0,0
5,1697606165,0.099609,-1.242249,-0.221252,16.453125,-6.171875,-1.015625,s1,ascending,breathingNormal,0,0,0
6,1697606205,0.238770,-1.436584,-0.275452,13.312500,-8.796875,6.906250,s1,ascending,breathingNormal,0,0,0
7,1697606245,0.625000,-1.344299,-0.260315,8.843750,-16.750000,8.406250,s1,ascending,breathingNormal,0,0,0
8,1697606285,-0.001465,-0.848938,-0.085266,-9.062500,-7.078125,5.781250,s1,ascending,breathingNormal,0,0,0
9,1697606325,0.411377,-0.892395,-0.272278,7.312500,-19.015625,5.421875,s1,ascending,breathingNormal,0,0,0


# Task 1

In [3]:
breatheNormalDataset = dataset[dataset.sub_activity == 'breathingNormal']
breatheNormalDataset.head()

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,student_number,activity,sub_activity,is_stationary,activity_label,sub_activity_label
0,1697605965,0.011963,-0.855774,-0.029846,-9.625000,-0.859375,-5.656250,s1,ascending,breathingNormal,0,0,0
1,1697606005,-0.001709,-0.826233,-0.036194,0.359375,4.953125,-1.656250,s1,ascending,breathingNormal,0,0,0
2,1697606045,-0.058838,-0.933899,-0.032532,3.437500,9.406250,-2.031250,s1,ascending,breathingNormal,0,0,0
3,1697606085,-0.002441,-1.115051,-0.028870,3.078125,6.921875,-5.984375,s1,ascending,breathingNormal,0,0,0
4,1697606125,-0.036621,-1.035217,-0.076477,4.328125,8.125000,0.625000,s1,ascending,breathingNormal,0,0,0


In [24]:
def sliding_window(X):
    step = 10
    segments = []
    labels = []

    for i in range(0, len(X)-SLIDING_WINDOW_LENGTH, step):
        xs = X['accel_x'].values[i: i+SLIDING_WINDOW_LENGTH]
        ys = X['accel_y'].values[i: i+SLIDING_WINDOW_LENGTH]
        zs = X['accel_z'].values[i: i+SLIDING_WINDOW_LENGTH]

        label = stats.mode(X['activity_label'][i: i+SLIDING_WINDOW_LENGTH])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)
    
    reshaped_segments = np.asarray(segments)  #.reshape(-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)
    labels = np.asarray(pd.get_dummies(labels), dtype=np.float32)
    return reshaped_segments, labels

batch_size = 128
accuracies = []
# for student in students:
student = students[5]

X_train, y_train = sliding_window(breatheNormalDataset)
X_test, y_test = sliding_window(breatheNormalDataset[breatheNormalDataset.student_number == student])

In [36]:
def build_model(X_train):
    output_dim = 12
    dropout_rate = 0.5

    input_shape=(SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)

    # LSTM parameters
    n_neurons_lstm = 128
    n_neurons = 64

    # optimization parameters
    lr = 0.001
    decay = 1e-6
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = Sequential()
    
    model.add(normalizer)

    model.add(Bidirectional(LSTM(units = n_neurons_lstm), input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    model.add(Dense(output_dim, activation='softmax'))

    adam = Adam(learning_rate=lr)

    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    model.summary()
    
    return model

In [25]:
def build_model_temp():
    batch_size = 128
    output_dim = 12
    epochs = 100
    dropout_rate = 0.1

    input_shape=(SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH)

    # LSTM parameters
    n_neurons_lstm = 256
    n_neurons = 64

    # optimization parameters
    lr = 0.0015
    decay = 1e-6

    model = Sequential()

    model.add(Bidirectional(LSTM(units = n_neurons_lstm, activation='relu',
                                 kernel_regularizer=l2(0.000002), bias_regularizer=l2(0.000002)), input_shape=(3,100)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(n_neurons, activation='relu', kernel_regularizer=l2(0.000002), bias_regularizer=l2(0.000002)))

    model.add(Dense(output_dim, activation='softmax', kernel_regularizer=l2(0.000002), bias_regularizer=l2(0.000002)))

    adam = Adam(learning_rate=lr, weight_decay=decay)

    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    model.summary()
    
    return model

In [26]:
model = build_model_temp()

history = model.fit(X_train, y_train, epochs = 100, validation_split = 0.30, batch_size = batch_size, verbose = 1)
accuracy = model.evaluate(X_test, y_test, batch_size = 32, verbose = 1)
accuracies.append(accuracy[1])

plt.figure(figsize=(15,10))
plt.plot(np.array(history.history['loss']), "r--", label = "Train loss")
plt.plot(np.array(history.history['accuracy']), "g--", label = "Train accuracy")
plt.plot(np.array(history.history['val_loss']), "r-", label = "Validation loss")
plt.plot(np.array(history.history['val_accuracy']), "g-", label = "Validation accuracy")
plt.title("Training session's progress over iterations")
plt.legend(loc='lower left')
plt.ylabel('Training Progress (Loss/Accuracy)')
plt.xlabel('Training Epoch')
# plt.ylim(0)
plt.grid()
plt.show()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirecti  (None, 512)               731136    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 12)                780       
                                                                 
Total params: 764748 (2.92 MB)
Trainable params: 764748 (2.92 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
350/350 [==============================] - 9s 17ms/step - loss: 0.8107 -

350/350 [==============================] - 6s 16ms/step - loss: 0.0619 - accuracy: 0.9776 - val_loss: 2.0103 - val_accuracy: 0.7998
Epoch 51/100
350/350 [==============================] - 6s 16ms/step - loss: 0.0612 - accuracy: 0.9773 - val_loss: 2.2155 - val_accuracy: 0.7859
Epoch 52/100
350/350 [==============================] - 6s 16ms/step - loss: 0.0585 - accuracy: 0.9785 - val_loss: 2.1395 - val_accuracy: 0.7929
Epoch 53/100
 45/350 [==>...........................] - ETA: 4s - loss: 0.0592 - accuracy: 0.9785

KeyboardInterrupt: 